In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [2]:
#pip install tensorflow==2.2.0

In [3]:
from loaders.Loaders import CSVLoader
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from featureSelection.baseFeatureSelector import LowVarianceFS
from splitters.splitters import SingletaskStratifiedSplitter
from models.kerasModels import KerasModel
from metrics.Metrics import Metric
from metrics.metricsFunctions import roc_auc_score, precision_score, accuracy_score, confusion_matrix, classification_report


import tensorflow as tf
print(tf.version.VERSION)
from tensorflow.keras.losses import binary_crossentropy

2.2.0


In [4]:
#Load Dataset
dataset = CSVLoader(dataset_path='preprocessed_dataset_wfoodb.csv', 
                    mols_field='Smiles', 
                    labels_fields='Class', 
                    id_field='ID')#, shard_size=4000)
dataset = dataset.create_dataset()
print(dataset.get_shape())

Mols_shape:  23290
Features_shape:  X not defined!
Labels_shape:  (23290,)
None


In [5]:
#Featurization
dataset = MorganFingerprint().featurize(dataset)
dataset.get_shape()

Featurizing datapoint 0
Featurizing datapoint 1000
Featurizing datapoint 2000
Featurizing datapoint 3000
Featurizing datapoint 4000
Featurizing datapoint 5000
Featurizing datapoint 6000


RDKit ERROR: [11:02:31] Explicit valence for atom # 1 Cl, 4, is greater than permitted


error in smile: O=[Cl]=O
Featurizing datapoint 7000


RDKit ERROR: [11:02:34] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [11:02:34] Explicit valence for atom # 1 Cl, 9, is greater than permitted


error in smile: OB1O[B]2(O)OB(O)O[B](O)(O1)O2
error in smile: O=[Cl-](=O)(=O)=O
Featurizing datapoint 8000
Featurizing datapoint 9000
Featurizing datapoint 10000
Featurizing datapoint 11000
Featurizing datapoint 12000
Featurizing datapoint 13000
Featurizing datapoint 14000


RDKit ERROR: [11:02:50] Explicit valence for atom # 0 P, 11, is greater than permitted


error in smile: [P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C
Featurizing datapoint 15000
Featurizing datapoint 16000
Featurizing datapoint 17000
Featurizing datapoint 18000
Featurizing datapoint 19000
Featurizing datapoint 20000
Featurizing datapoint 21000
Featurizing datapoint 22000
Featurizing datapoint 23000
Elements with indexes:  [6257, 7708, 7709, 14244]  were removed due to the presence of NAs!
The elements in question are:  ['O=[Cl]=O' 'OB1O[B]2(O)OB(O)O[B](O)(O1)O2' 'O=[Cl-](=O)(=O)=O'
 '[P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C']
Mols_shape:  23286
Features_shape:  (23286, 1024)
Labels_shape:  (23286,)


In [6]:
#Feature Selection
dataset = LowVarianceFS(0.15).featureSelection(dataset)
dataset.get_shape()

Mols_shape:  23286
Features_shape:  (23286, 49)
Labels_shape:  (23286,)


In [7]:
#Data Split
splitter = SingletaskStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, frac_train=0.6, 
                                                                             frac_valid=0.2, frac_test=0.2)

In [8]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np


input_dim = train_dataset.X.shape[1]


def create_model(optimizer='adam', dropout=0.5, input_dim=input_dim):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [9]:
#print(type(keras_model))
#from sklearn.model_selection import GridSearchCV 
#model = KerasClassifier(build_fn=create_model, epochs=15, batch_size=10, verbose=1, optimizer='adam')

#model.fit(train_dataset.features, train_dataset.y)
#params={'optimizer':['adam', 'rsmprop'],  
#        'nb_epoch':[200, 100, 300, 400]}

#gs=GridSearchCV(estimator=model, param_grid=params, cv=3) 
# now fit the dataset to the GridSearchCV object.  
#gs = gs.fit(train_dataset.features, train_dataset.y)

#print(gs)
#print(type(model))

#seed = 7
#np.random.seed(seed)
#print(1)
#kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
#print(2)
#results = cross_val_score(model, train_dataset.features, train_dataset.y, cv=kfold)
#print(3)
#print(results)

#model = KerasModel(keras_model, binary_crossentropy)

In [10]:
from models.kerasModels import KerasModel

input_dim = train_dataset.X.shape[1]
print(input_dim)
model = KerasModel(create_model, epochs = 5, verbose=1, optimizer='adam')#, input_dim=input_dim)



49


In [11]:
print(train_dataset.X.shape, train_dataset.y.shape)


model.fit(train_dataset)#.features, train_dataset.y)#, nb_epoch=50)


(13968, 49) (13968,)
kerasModels fit!
Epoch 1/5
1397/1397 [==============================] - 2s 1ms/step - loss: 0.2532 - accuracy: 0.9253
Epoch 2/5
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1845 - accuracy: 0.9439
Epoch 3/5
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1732 - accuracy: 0.9450
Epoch 4/5
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1714 - accuracy: 0.9451
Epoch 5/5
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1687 - accuracy: 0.9459


In [12]:
metrics = [Metric(roc_auc_score), 
           Metric(precision_score), 
           Metric(accuracy_score), 
           Metric(confusion_matrix), 
           Metric(classification_report)]


print('training set score:', model.evaluate(train_dataset, metrics))
print('test set score:', model.evaluate(test_dataset, metrics))

Instructions for updating:
Please use `model.predict()` instead.
1397/1397 [==============================] - 1s 718us/step
roc_auc_score: 
 0.5416620865606853
precision_score: 
 0.7613636363636364
accuracy_score: 
 0.9468069873997709
confusion_matrix: 
 [[13158    21]
 [  722    67]]
classification_report: 
               precision    recall  f1-score   support

           0       0.95      1.00      0.97     13179
           1       0.76      0.08      0.15       789

    accuracy                           0.95     13968
   macro avg       0.85      0.54      0.56     13968
weighted avg       0.94      0.95      0.93     13968

training set score: {'roc_auc_score': 0.5416620865606853, 'precision_score': 0.7613636363636364, 'accuracy_score': 0.9468069873997709, 'confusion_matrix': 3492.0, 'classification_report': None}
466/466 [==============================] - 0s 743us/step
roc_auc_score: 
 0.5462765253051216
precision_score: 
 0.6944444444444444
accuracy_score: 
 0.946520618556701
c

In [13]:
from parameterOptimization.HyperparameterOpt import HyperparamOpt_Valid, HyperparamOpt_CV
#Hyperparameter Optimization
optimizer = HyperparamOpt_Valid(create_model)

params_dict = {'optimizer' : ['adam', 'rmsprop'],
              'dropout' : [0.2, 0.4, 0.5]}

best_model, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict, train_dataset, 
                                                                        valid_dataset, Metric(roc_auc_score))

print('#################')
print(best_hyperparams)
print(best_model)

#Evaluate model
best_model.evaluate(test_dataset, Metric(roc_auc_score)) #check error when using more metrics

MODE:  classification
Fitting 6 random models from a space of 6 possible models.
Fitting model 1/6
hyperparameters: {'optimizer': 'adam', 'dropout': 0.2}
437/437 [==============================] - 1s 1ms/step - loss: 0.2465 - accuracy: 0.9345
expected str, bytes or os.PathLike object, not NoneType
roc_auc_score: 
 0.7707112967102722
Model 1/6, Metric roc_auc_score, Validation set 1: 0.770711
	best_validation_score so far: 0.770711
Fitting model 2/6
hyperparameters: {'optimizer': 'adam', 'dropout': 0.4}
437/437 [==============================] - 1s 1ms/step - loss: 0.2394 - accuracy: 0.9401
expected str, bytes or os.PathLike object, not NoneType
roc_auc_score: 
 0.7853009259259258
Model 2/6, Metric roc_auc_score, Validation set 2: 0.785301
	best_validation_score so far: 0.785301
Fitting model 3/6
hyperparameters: {'optimizer': 'adam', 'dropout': 0.5}
437/437 [==============================] - 1s 1ms/step - loss: 0.2328 - accuracy: 0.9411
expected str, bytes or os.PathLike object, not No

{'roc_auc_score': 0.7826329305436666}

In [14]:
#Hyperparameter Optimization with CV
optimizer = HyperparamOpt_CV(create_model)

best_rf, best_hyperparams, all_results = optimizer.hyperparam_search('keras',
                                                                     params_dict, 
                                                                     train_dataset,  
                                                                     'accuracy', 
                                                                     cv=3,
                                                                     n_iter_search=10)

print('#################')
print(best_hyperparams)
print(best_rf)
#Evaluate model
best_rf.evaluate(test_dataset, metrics) #solve problem with kerasclassifier/regressor and evaluate method 

MODEL TYPE:  keras
291/291 [==============================] - 0s 1ms/step - loss: 0.3880 - accuracy: 0.8407
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
437/437 [==============================] - 1s 1ms/step - loss: 0.2529 - accuracy: 0.9301

 
 Best accuracy: 0.943514 using {'dropout': 0.2, 'optimizer': 'adam'}

 accuracy: 0.943514 (0.000000) with: {'dropout': 0.2, 'optimizer': 'adam'} 


 accuracy: 0.943514 (0.000000) with: {'dropout': 0.2, 'optimizer': 'rmsprop'} 


 accuracy: 0.943514 (0.000000) with: {'dropout': 0.4, 'optimizer': 'adam'} 


 accuracy: 0.943514 (0.000000) with: {'dropout': 0.4, 'optimizer': 'rmsprop'} 


 accuracy: 0.943514 (0.000000) with: {'dropout': 0.5, 'optimizer': 'adam'} 




/opt/conda/lib/python3.6/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


precision_score: 
 0.5102040816326531
accuracy_score: 
 0.9441580756013745
confusion_matrix: 
 [[4321   72]
 [ 188   75]]
classification_report: 
               precision    recall  f1-score   support

           0       0.96      0.98      0.97      4393
           1       0.51      0.29      0.37       263

    accuracy                           0.94      4656
   macro avg       0.73      0.63      0.67      4656
weighted avg       0.93      0.94      0.94      4656



{'roc_auc_score': 0.6343906958789434,
 'precision_score': 0.5102040816326531,
 'accuracy_score': 0.9441580756013745,
 'confusion_matrix': 1164.0,
 'classification_report': None}

In [15]:
#pip install tensorflow==2.2.0

In [16]:

#import tensorflow as tf
#tf.version.VERSION

In [17]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])